In [ ]:
import subprocess
import json
import pandas as pd
import os

offered_feeds = pd.read_csv('statsperform_feeds.csv')

{
    k:v for k, v in zip(offered_feeds.FeedCode, offered_feeds.FeedWidgetName)
}

In [ ]:
def _access_statsperform_api(feed_name: str,
                             tourney_cal_id: str = None,
                             match_id: str = None
                             ):

    proxy_url = "http://127.0.0.1:3128"
    statsperform_base_url = 'https://api.performfeeds.com/soccerdata'
    auth_key = str(os.environ['STATSPERFORM_API_KEY'])

    master_dict = {
        '_rt':'b',
        '_fmt':'json'
    }

    if feed_name == 'match':
        assert tourney_cal_id is not None, "To access match feed data, a tournament calendar ID must be passed in."
        master_dict['tmcl'] = tourney_cal_id
        master_dict['_pgSz'] = 1000
    if feed_name == 'matchstats':
        assert match_id is not None, "To access match stats feed, a match ID must be passed in."
        master_dict['fx'] = match_id
        master_dict['detailed'] = 'yes'
        master_dict['people'] = 'yes'

    query_string = '&'.join([f'{k}={v}' for k, v in master_dict.items()])

    q_command = f""" curl -x "{proxy_url}" '{statsperform_base_url}/{feed_name}/{auth_key}/authorized?&{query_string}' """ if feed_name == 'tournamentcalendar' else \
        f""" curl -x "{proxy_url}" '{statsperform_base_url}/{feed_name}/{auth_key}?&{query_string}' """

    print(q_command)

    process = subprocess.run(
                q_command,
                shell=True,
                capture_output=True,
                check=False # Set to True if you want a CalledProcessError for non-zero exit codes
            )

    stdout = process.stdout
    json_output = json.loads(stdout)
    return json_output


## tournaments

In [ ]:
# Get all available tournament calendar IDs with OT2 feed
feed = 'tournamentcalendar'
comps = _access_statsperform_api(feed)
competitions = pd.DataFrame(comps['competition'])
# competitions

In [ ]:
# competition id for the premier league
comp_id = "2kwbbcootiqqgmrzs6o5inle5"
pd.DataFrame(competitions[competitions['id'] == comp_id].tournamentCalendar.iloc[0])

In [ ]:
# tournament calendar id for the 24/25 premier league season
tourney_cal_id = "9n12waklv005j8r32sfjj2eqc"

## matches

In [ ]:
# Get all matches with MA1 feed

french_matches = _access_statsperform_api(feed_name='match',
                         tourney_cal_id=tourney_cal_id)

all_matches = pd.DataFrame([x['matchInfo'] for x in french_matches['match']])
all_matches

In [ ]:
# getting access to player data using MA2 feed

example_match = '7rd4o9m0otkg7u58cqplq19uc'

example_match_request = _access_statsperform_api(feed_name='matchstats',
                         query_paramo={'_rt':'b',
                          '_fmt':'json',
                          'fx': example_match,
                          # '_pgSz':1000,
                          'detailed':'yes',
                          'people':'yes'})

match_data = pd.DataFrame(example_match_request['liveData']['lineUp'])

dembele = pd.DataFrame(match_data.iloc[1].player).iloc[13]

pd.Series({x['type']:x['value'] for x in dembele.stat})